# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [37]:
# import libraries
import nltk
nltk.download(['stopwords','punkt', 'wordnet', 'averaged_perceptron_tagger'])# download for lemmatization

import numpy as np
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, \
    classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.svm import LinearSVC

import pickle
import datetime

from sqlalchemy import create_engine

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\crist\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [24]:
# load data from database
engine = create_engine('sqlite:///DisasterResponseDatabase.db')

df = pd.read_sql_table('MessageClassification',con='sqlite:///DisasterResponseDatabase.db')
X = df.iloc[:,1] #message
Y = df.iloc[:,4:] #aid_centers, aid_related, buildings, ... 

In [25]:
X.sample(5)

17997    Air Serv International, a non-profit organizat...
23241    The Pakistan police advised on November 26th t...
6748                I know speak english can I get a job? 
11485    atiasdelrio Interseismic strain accumulation m...
10500    Oh boy , power just went in and out .. Indicat...
Name: message, dtype: object

In [26]:
Y.head()

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [27]:
def tokenize(text):
    text = text.lower() #makes text lower case
    text = re.sub(r"[^a-zA-Z0-9]"," ",text) #removes non-alphabetic characters
    words=word_tokenize(text) #splits into words

    words = [w for w in words if w not in stopwords.words("english")]# Remove stop words    
    words = [PorterStemmer().stem(w) for w in words] #apply stemming to words (eg. branching -> branch)
    words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]#apply lemmatizing to words (eg. was -> is)
    
    return words

In [28]:
#test
print("text:",X[1],"\ntokens:", tokenize(X[1]))

text: Is the Hurricane over or is it not over 
tokens: ['hurrican']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [29]:
#create pipeline

pipeline1 = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer()), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1)) #create the Classifier object
]) 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
#perform train / test split
X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), Y.as_matrix(), test_size=0.2, random_state=42)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [31]:
#Train pipeline
pipeline1.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [32]:
# test the classifier
y_pred = pipeline1.predict(X_test)

In [33]:
#Convert prediction numpy into dataframe
y_test=pd.DataFrame(data=y_test,columns=Y.columns)
y_pred=pd.DataFrame(data=y_pred,columns=Y.columns)


In [34]:
accuracy = (y_pred == y_test).mean()

print("Accuracy:\n",accuracy)

Accuracy:
 aid_centers               0.987364
aid_related               0.734061
buildings                 0.951177
child_alone               1.000000
clothing                  0.987747
cold                      0.980088
death                     0.960942
direct_report             0.844342
earthquake                0.965537
electricity               0.979897
fire                      0.990044
floods                    0.942562
food                      0.944859
hospitals                 0.989087
infrastructure_related    0.933563
medical_help              0.919778
medical_products          0.953666
military                  0.970898
missing_people            0.989470
money                     0.978556
offer                     0.995788
other_aid                 0.861957
other_infrastructure      0.955964
other_weather             0.943711
refugees                  0.968217
related                   0.801647
request                   0.880720
search_and_rescue         0.971855
security 

In [35]:
results_dic1={}
for column in y_pred.columns:
    print(column)
    print(classification_report(y_test[column], y_pred[column]))
    print('_____________________________________________________')

    results_dic1[column]=classification_report(y_test[column], y_pred[column],output_dict=True)['weighted avg']

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5158
           1       0.00      0.00      0.00        65

   micro avg       0.99      0.99      0.99      5223
   macro avg       0.49      0.50      0.50      5223
weighted avg       0.98      0.99      0.98      5223

_____________________________________________________
aid_related
              precision    recall  f1-score   support

           0       0.74      0.85      0.79      3037
           1       0.73      0.58      0.64      2186

   micro avg       0.73      0.73      0.73      5223
   macro avg       0.73      0.71      0.72      5223
weighted avg       0.73      0.73      0.73      5223

_____________________________________________________
buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4962
           1       0.61      0.07      0.12       261

   micro avg       0.95      0.95      0

C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


other_aid
              precision    recall  f1-score   support

           0       0.87      0.99      0.93      4503
           1       0.49      0.04      0.08       720

   micro avg       0.86      0.86      0.86      5223
   macro avg       0.68      0.52      0.50      5223
weighted avg       0.82      0.86      0.81      5223

_____________________________________________________
other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4996
           1       0.20      0.00      0.01       227

   micro avg       0.96      0.96      0.96      5223
   macro avg       0.58      0.50      0.49      5223
weighted avg       0.92      0.96      0.94      5223

_____________________________________________________
other_weather
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4931
           1       0.48      0.07      0.12       292

   micro avg       0.94      

### 6. Improve your model
Use grid search to find better parameters. 

In [38]:
start_time  = datetime.datetime.now()

In [39]:
pipeline2 = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer()), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(RandomForestClassifier())) #create the Classifier object
]) 


In [40]:
pipeline2.get_params().keys()

dict_keys(['memory', 'steps', 'vectorizer', 'tfidf', 'clf', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__preprocessor', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs

In [41]:
parameters = {    
        'tfidf__use_idf': (True, False), 
        'clf__estimator__n_estimators': [50, 100],
        'clf__estimator__min_samples_split': [2,4] }

In [42]:
grid_cv = GridSearchCV(pipeline2, param_grid=parameters, cv=5,verbose=3,n_jobs=-1)

In [ ]:
grid_cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 38.2min


In [ ]:
end_time = datetime.datetime.now()
print ('Select Done..., Time Cost: %d' % ((end_time - start_time).seconds))

In [ ]:
grid_cv.best_score_

In [ ]:
best_parameters=grid_cv.best_estimator_.get_params()

In [ ]:
best_parameters

In [ ]:
#https://www.programcreek.com/python/example/91151/sklearn.model_selection.GridSearchCV

for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
pipeline2 = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer(use_idf=True)), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split= 4,n_estimators= 100),\
        n_jobs=-1)) #create the Classifier object
]) 

In [ ]:
pipeline2.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# test the classifier
y_pred = pipeline2.predict(X_test)

In [ ]:
#Convert prediction numpy into dataframe
y_test=pd.DataFrame(data=y_test,columns=Y.columns)
y_pred=pd.DataFrame(data=y_pred,columns=Y.columns)

In [ ]:
results_dic2={}

for column in y_pred.columns:
    print(column)
    print(classification_report(y_test[column], y_pred[column]))
    print('_____________________________________________________')
    
    results_dic2[column]=classification_report(y_test[column], y_pred[column],output_dict=True)['weighted avg']


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
start_time  = datetime.datetime.now()

In [ ]:
pipeline3 = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer()), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(LinearSVC())) #create the Classifier object
]) 

In [ ]:
pipeline3.get_params()

In [ ]:
parameters = {    
        'tfidf__use_idf': (True, False), 
        'clf__estimator__C': [0.1, 1,10,100,1000,
  }

In [ ]:
grid_cv = GridSearchCV(pipeline3, param_grid=parameters, cv=5,verbose=3,n_jobs=-1)

In [ ]:
grid_cv.fit(X_train, y_train)

In [ ]:
end_time = datetime.datetime.now()
print ('Select Done..., Time Cost: %d' % ((end_time - start_time).seconds))

In [ ]:
grid_cv.best_score_

In [ ]:
best_parameters=grid_cv.best_estimator_.get_params()

In [ ]:
best_parameters

In [ ]:
#https://www.programcreek.com/python/example/91151/sklearn.model_selection.GridSearchCV

for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
pipeline3 = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer=tokenize)), #create the CountVectorizer object
    ('tfidf', TfidfTransformer(use_idf=True)), #create Tfidftransformer object    
    ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split= 4,n_estimators= 100),\
        n_jobs=-1)) #create the Classifier object
]) 

In [ ]:
pipeline3.fit(X_train, y_train)

In [ ]:
# test the classifier
y_pred = pipeline3.predict(X_test)

In [ ]:
#Convert prediction numpy into dataframe
y_test=pd.DataFrame(data=y_test,columns=Y.columns)
y_pred=pd.DataFrame(data=y_pred,columns=Y.columns)

In [ ]:
results_dic3={}

for column in y_pred.columns:
    print(column)
    print(classification_report(y_test[column], y_pred[column]))
    print('_____________________________________________________')
    
    results_dic3[column]=classification_report(y_test[column], y_pred[column],output_dict=True)['weighted avg']


### 9. Export your model as a pickle file

In [ ]:
pickle_out = open("model.pickle","wb")
pickle.dump(pipeline, pickle_out)

pickle_out.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.